In [1]:
import pandas as pd
import numpy as np
import dask #used here to scale computing capabilities of pandas through parallelism
import dask.dataframe as dd
import pickle #since openning the data was time consuming let's just save it in a pickle file

In [5]:
crsp=dd.read_csv("CRSP_M_19502020.csv")

In [6]:
ibes=dd.read_csv("IBES_Q_19802020.csv")

In [7]:
comp=dd.read_csv("Compustat_Q_19712020.csv")

In [8]:
drp_crsp=['DIVAMT','EXCHCD','FACPR', 'NWPERM', 'BIDLO', 'ASKHI','TICKER','BID','ASK','SPREAD','RETX','SHRCD','Unnamed: 0']
crsp=crsp.drop(columns=drp_crsp)

In [9]:
crsp

,PERMNO,date,NCUSIP,CUSIP,PRC,VOL,RET,SHROUT
npartitions=2,,,,,,,,
,int64,object,object,object,float64,float64,float64,float64
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


In [10]:
crsp['PRC']=crsp['PRC'].abs()

In [11]:
drop_comp=['indfmt', 'consol','curcdq','popsrc','datafmt','tic','datacqtr', 'datafqtr', 'actq','ceqq','dlcq', 'dlttq','exchg', 'costat']
comp=comp.drop(columns=drop_comp)

In [12]:
comp

,Unnamed: 0,gvkey,datadate,fyearq,fqtr,fyr,cusip,atq,cshoq,ibq,revtq,prccq,naics,sic
npartitions=2,,,,,,,,,,,,,,
,int64,int64,object,int64,int64,int64,object,float64,float64,float64,float64,float64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [13]:
#We have to drop he last digit in CUSIP for Compustat to be equal to CRSP
comp['cusip'] = comp['cusip'].str[:-1]
comp

,Unnamed: 0,gvkey,datadate,fyearq,fqtr,fyr,cusip,atq,cshoq,ibq,revtq,prccq,naics,sic
npartitions=2,,,,,,,,,,,,,,
,int64,int64,object,int64,int64,int64,object,float64,float64,float64,float64,float64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [14]:
drop_ibes=['Unnamed: 0','ESTIMATOR','FPI','REVDATS','REVTIMS']
ibes=ibes.drop(columns=drop_ibes)
ibes

,TICKER,CUSIP,ANALYS,VALUE,FPEDATS,ANNDATS,ANNTIMS,ACTUAL,ANNDATS_ACT,ANNTIMS_ACT
npartitions=3,,,,,,,,,,
,object,object,int64,float64,object,object,object,float64,object,object
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


In [15]:
#Celaning out the date that comes as a data reading error when we read a stata file
ibes['ANNTIMS']=ibes['ANNTIMS'].str[10:]
ibes['ANNTIMS_ACT']=ibes['ANNTIMS_ACT'].str[10:]
ibes

,TICKER,CUSIP,ANALYS,VALUE,FPEDATS,ANNDATS,ANNTIMS,ACTUAL,ANNDATS_ACT,ANNTIMS_ACT
npartitions=3,,,,,,,,,,
,object,object,int64,float64,object,object,object,float64,object,object
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


In [16]:
#transform both columns into datetime.date objects
ibes['ANNDATS']=dd.to_datetime(ibes['ANNDATS'])
ibes['ANNDATS_ACT']=dd.to_datetime(ibes['ANNDATS_ACT'])
ibes['FPEDATS']=dd.to_datetime(ibes['FPEDATS'])

In [17]:
#the date.days attribute will return the days in int format
ibes_fltr = ibes[(ibes['ANNDATS']-ibes['ANNDATS_ACT']).dt.days >= -15]

In [18]:
ibes_fltr_grp=ibes_fltr.groupby(['TICKER','FPEDATS','ANALYS'])
ibes_fin_fltr=ibes_fltr_grp['ANNDATS'].max()
ibes_fin_fltr

Dask Series Structure:
npartitions=1
    datetime64[ns]
               ...
Name: ANNDATS, dtype: datetime64[ns]
Dask Name: series-groupby-max-agg, 73 tasks

In [19]:
df1=ibes_fin_fltr.reset_index()

In [20]:
df2=df1.merge(ibes_fltr)

In [21]:
df2

,TICKER,FPEDATS,ANALYS,ANNDATS,CUSIP,VALUE,ANNTIMS,ACTUAL,ANNDATS_ACT,ANNTIMS_ACT
npartitions=3,,,,,,,,,,
,object,datetime64[ns],int64,datetime64[ns],object,float64,object,float64,datetime64[ns],object
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


In [22]:
df2=df2.compute()

In [23]:
ibes_fnl=df2.groupby(['TICKER','CUSIP','FPEDATS','ANNDATS_ACT','ANNTIMS_ACT','ACTUAL']).agg({'VALUE':'median'})

In [24]:
ibes_fnl

VALUE
TICKER CUSIP    FPEDATS    ANNDATS_ACT ANNTIMS_ACT ACTUAL       
0000   87482X10 2014-03-31 2014-05-06   10:45:00    0.12   0.100
                2014-06-30 2014-08-06   17:05:00    0.27   0.190
0001   26878510 2013-12-31 2014-02-27   23:13:00    0.20   0.230
                2014-03-31 2014-05-07   16:15:00    0.18   0.170
                2014-06-30 2014-08-06   17:10:00    0.23   0.200
...                                                          ...
CC     17273710 2007-08-31 2007-09-20   07:55:00   -0.38  -0.130
                2007-11-30 2007-12-21   07:55:00   -0.64  -0.340
                2008-02-29 2008-04-09   07:55:00    0.10  -0.105
                2008-05-31 2008-06-19   07:55:00   -1.00  -0.970
                2008-08-31 2008-09-29   07:55:00   -1.00  -1.010

[31396 rows x 1 columns]

In [25]:
ibes_fnl=ibes_fnl.reset_index()
ibes_fnl

,TICKER,CUSIP,FPEDATS,ANNDATS_ACT,ANNTIMS_ACT,ACTUAL,VALUE
0,0000,87482X10,2014-03-31,2014-05-06,10:45:00,0.12,0.100
1,0000,87482X10,2014-06-30,2014-08-06,17:05:00,0.27,0.190
2,0001,26878510,2013-12-31,2014-02-27,23:13:00,0.20,0.230
3,0001,26878510,2014-03-31,2014-05-07,16:15:00,0.18,0.170
4,0001,26878510,2014-06-30,2014-08-06,17:10:00,0.23,0.200
...,...,...,...,...,...,...,...
31391,CC,17273710,2007-08-31,2007-09-20,07:55:00,-0.38,-0.130
31392,CC,17273710,2007-11-30,2007-12-21,07:55:00,-0.64,-0.340
31393,CC,17273710,2008-02-29,2008-04-09,07:55:00,0.10,-0.105
31394,CC,17273710,2008-05-31,2008-06-19,07:55:00,-1.00,-0.970


In [26]:
ibes_fnl['SURPRISE']=ibes_fnl['ACTUAL']-ibes_fnl['VALUE']
ibes_fnl

,TICKER,CUSIP,FPEDATS,ANNDATS_ACT,ANNTIMS_ACT,ACTUAL,VALUE,SURPRISE
0,0000,87482X10,2014-03-31,2014-05-06,10:45:00,0.12,0.100,0.020
1,0000,87482X10,2014-06-30,2014-08-06,17:05:00,0.27,0.190,0.080
2,0001,26878510,2013-12-31,2014-02-27,23:13:00,0.20,0.230,-0.030
3,0001,26878510,2014-03-31,2014-05-07,16:15:00,0.18,0.170,0.010
4,0001,26878510,2014-06-30,2014-08-06,17:10:00,0.23,0.200,0.030
...,...,...,...,...,...,...,...,...
31391,CC,17273710,2007-08-31,2007-09-20,07:55:00,-0.38,-0.130,-0.250
31392,CC,17273710,2007-11-30,2007-12-21,07:55:00,-0.64,-0.340,-0.300
31393,CC,17273710,2008-02-29,2008-04-09,07:55:00,0.10,-0.105,0.205
31394,CC,17273710,2008-05-31,2008-06-19,07:55:00,-1.00,-0.970,-0.030


In [27]:
ibes_filtered=open("ibes_filtered.pickle","wb")

In [28]:
pickle.dump(ibes_fnl,ibes_filtered)
ibes_filtered.close()

In [29]:
ibes_fltr_dask=open("ibes_filtered_dask.pickle","wb")

In [31]:
sd=dd.from_pandas(ibes_fnl,npartitions=15)

In [32]:
pickle.dump(sd,ibes_fltr_dask)

In [33]:
ibes_fltr_dask.close()

In [34]:
crsp=crsp.compute()

In [35]:
crsp

,PERMNO,date,NCUSIP,CUSIP,PRC,VOL,RET,SHROUT
0,10000,1985-12-31,NaN,68391610,NaN,NaN,NaN,NaN
1,10000,1986-01-31,68391610,68391610,4.375000,1771.0,NaN,3680.0
2,10000,1986-02-28,68391610,68391610,3.250000,828.0,-0.257143,3680.0
3,10000,1986-03-31,68391610,68391610,4.437500,1078.0,0.365385,3680.0
4,10000,1986-04-30,68391610,68391610,4.000000,957.0,-0.098592,3793.0
...,...,...,...,...,...,...,...,...
357355,18004,2018-11-30,46139W80,46139W80,24.730000,0.0,0.004189,2000.0
357356,18004,2018-12-31,46139W80,46139W80,24.922800,37.0,0.012748,2000.0
357357,18004,2018-12-31,46139W80,46139W80,24.922800,37.0,0.012748,2000.0
357358,18004,2019-01-31,46139W80,46139W80,25.240000,13.0,0.015937,2000.0


In [36]:
crsp_filtered=open("crsp_filtered_dask.pickle","wb")

In [37]:
pickle.dump(dd.from_pandas(crsp,npartitions=15),crsp_filtered)

In [38]:
crsp_filtered.close()

In [39]:
comp_filtered=open("comp_filtered_dask.pickle","wb")

In [41]:
pickle.dump(comp,comp_filtered)

In [42]:
comp_filtered.close()

In [43]:
ibes_crsp=ibes_fnl.merge(crsp,left_on='CUSIP', right_on='NCUSIP')

In [44]:
ibes_crsp

,TICKER,CUSIP_x,FPEDATS,ANNDATS_ACT,ANNTIMS_ACT,ACTUAL,VALUE,SURPRISE,PERMNO,date,NCUSIP,CUSIP_y,PRC,VOL,RET,SHROUT
0,0000,87482X10,2014-03-31,2014-05-06,10:45:00,0.12,0.10,0.02,14471,2014-02-28,87482X10,87482X10,13.880000,109117.0,NaN,69943.0
1,0000,87482X10,2014-03-31,2014-05-06,10:45:00,0.12,0.10,0.02,14471,2014-03-31,87482X10,87482X10,14.640000,78851.0,0.054755,69962.0
2,0000,87482X10,2014-03-31,2014-05-06,10:45:00,0.12,0.10,0.02,14471,2014-04-30,87482X10,87482X10,13.410000,47374.0,-0.084016,69978.0
3,0000,87482X10,2014-03-31,2014-05-06,10:45:00,0.12,0.10,0.02,14471,2014-05-30,87482X10,87482X10,13.570000,33556.0,0.011931,69992.0
4,0000,87482X10,2014-03-31,2014-05-06,10:45:00,0.12,0.10,0.02,14471,2014-06-30,87482X10,87482X10,13.790000,40869.0,0.016212,70451.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345437,CBYL,48349710,2019-04-30,2019-07-15,07:30:00,-0.49,-0.28,-0.21,15285,2020-08-31,48349710,48349710,13.100000,14588.0,0.332655,17880.0
1345438,CBYL,48349710,2019-04-30,2019-07-15,07:30:00,-0.49,-0.28,-0.21,15285,2020-09-30,48349710,48349710,12.590000,9333.0,-0.038931,17908.0
1345439,CBYL,48349710,2019-04-30,2019-07-15,07:30:00,-0.49,-0.28,-0.21,15285,2020-10-30,48349710,48349710,17.209999,23210.0,0.366958,17916.0
1345440,CBYL,48349710,2019-04-30,2019-07-15,07:30:00,-0.49,-0.28,-0.21,15285,2020-11-30,48349710,48349710,18.709999,22268.0,0.087159,17916.0
